In [84]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [91]:
# FABRIC_LINE = "kona_cotton"
FABRIC_LINE = "artisan_batiks_prisma_dyes"

In [85]:
# Read fabric page from URL
BASE_URL = "https://www.robertkaufman.com"
URL = BASE_URL + "/fabrics/" + FABRIC_LINE + "/?page=All#products"
page = requests.get(URL)
soup = BeautifulSoup(page.content, "html.parser")
result = soup.find_all(class_="fabric-item-fabrics-list")

In [86]:
# Get all fabric items and filter out special products
fabrics = result[0].find_all("li")
fabrics = list(
    filter(
        lambda x: "specialproduct" not in x["id"] and "colorcard" not in x["id"],
        fabrics,
    )
)

In [87]:
# Get the image asset path
imagePath = "/".join(fabrics[-1].find("img")["src"].split("/")[:-1])
imagePath

'/assets/fabric/thumbs'

In [88]:
def extract_data(fabric):
    return [
        fabric.find("img")["name"],
        " ".join(fabric.find_all("div")[-1].text.strip().split()[3:]),
        fabric.find("img")["src"].split("/")[-1],
    ]


# Extract relevant data
df = pd.DataFrame(
    list(map(extract_data, fabrics)), columns=["id", "name", "src"]
)
df

id       name               src
0      AMD-7000-2      BLACK    AMD-7000-2.jpg
1      AMD-7000-3        RED    AMD-7000-3.jpg
2      AMD-7000-4       BLUE    AMD-7000-4.jpg
3      AMD-7000-8     ORANGE    AMD-7000-8.jpg
4      AMD-7000-9       NAVY    AMD-7000-9.jpg
..            ...        ...               ...
151  AMD-7000-470  HYDRANGEA  AMD-7000-470.jpg
152  AMD-7000-471   LARKSPUR  AMD-7000-471.jpg
153  AMD-7000-472     DAHLIA  AMD-7000-472.jpg
154  AMD-7000-473    BOUQUET  AMD-7000-473.jpg
155  AMD-7000-474   HONEYDEW  AMD-7000-474.jpg

[156 rows x 3 columns]

In [89]:
def download_image(src):
    data = requests.get(BASE_URL + imagePath + "/" + src).content
    with open("../web/images/" + FABRIC_LINE + "/" + src, "wb") as handler:
        handler.write(data)


# Download all the images
for src in df["src"]:
    download_image(src)

In [90]:
# Write data to csv
df.to_csv("../web/data/" + FABRIC_LINE + ".csv", index=False)